In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import datetime
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
# from ipynb.fs.full.Preprocessing import clean_data
%load_ext tensorboard

# df = pd.read_csv('./Data/songs_cleaned.csv', index_col='Unnamed: 0')
# print(df.shape)
# df.describe()

In [2]:
df = pd.read_csv('data/songs_cleaned.csv', index_col='Unnamed: 0')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/songs_cleaned.csv'

In [ ]:
# Simple autoencoder

encoded_dim = 16

input_song = Input(shape = (df.shape[1], ))

h1 = Dense(32, activation = 'relu')(input_song)

encoded = Dense(encoded_dim, activation = 'relu')(h1)

dh1 = Dense(32, activation='sigmoid')(encoded)

decoded = Dense(df.shape[1], activation='sigmoid')(dh1)

autoencoder = Model(input_song, decoded)

encoder = Model(input_song, encoded)

autoencoder.compile(optimizer='nadam', loss='mean_squared_error')

In [ ]:
stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3)
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

logdir = os.path.join("logs", f"SimpleAutoencoder-{now}")
tensorboard = TensorBoard(log_dir=logdir)

autoencoder.fit(df, # input image to encoder
                df, # provide input image to decoder so the model learns how to reconstruct the input image 
                epochs=100,
                validation_split=.2,
                callbacks=[stop, tensorboard])

### Shallow NN Summary:
0 hidden layers: Val Loss: 0.3160
1 hidden layer: 

In [ ]:
decoded_songs = autoencoder.predict(df)
encoded_songs = encoder.predict(df)

embeddings_df = pd.DataFrame(encoded_songs, index=df.index)

In [ ]:
print(embeddings_df.shape)
embeddings_df

In [ ]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=11, algorithm='ball_tree')
nn.fit(encoded_songs)

In [ ]:
len(encoded_songs[0])

In [ ]:
# Saving the encoded_songs embedding as a df and the Nearest Neighbours model for use in our app
import pickle

embeddings_df = pd.DataFrame(encoded_songs, index=df.index)

# Saving the embeddings to the disk
embeddings_df.to_csv('data/autoencoder_embeddings.csv')

# save the model to disk
filename = 'autoencoder_nn_model.sav'
pickle.dump(nn, open(filename, 'wb'))

# some time later...
 
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

In [ ]:
embeddings_df

In [ ]:
# Testing our model with multiple songs
raw_df = pd.read_csv('./Data/tracks.csv')

test_songs = {'California Love': '0LyLz6XsVs6wz85dK0S6EG', 'The Gambler': '5KqldkCunQ2rWxruMEtGh0', 
              'Eye of the Tiger': '2HHtWyy5CgaQbC7XSoOb0e', 'You are my Sunshine': '4ApTr4wbyOucy2lbAvyuuV'}

# Loop over all our test songs and get the recommendations
recommendations = {}

print('DeepSpot\'s recommendation system:\n\n')

for name, id in test_songs.items():
    query = raw_df.index[raw_df['id'] == id][0]
    test_song = pd.DataFrame([raw_df.iloc[query, :]])
    
    score, close_image_idx = nn.kneighbors(encoded_songs[query].reshape(1,-1))
    k_closest_songs_indexes = close_image_idx[0][1:]
    
    print('Top 10 recommendations for {} by {}:\n'.format(name, test_song.artists))
    for i in range(len(k_closest_songs_indexes)):
        song = raw_df.iloc[k_closest_songs_indexes[i], :]
        print('{} by {} with a score of {}\n'.format(song['name'], song.artists, score[0][1:][i]))
        print('URL: https://open.spotify.com/track/{}\n'.format(song['id']))
    print('___________________________________________________________________________________\n')
    
    recommendations[name] = k_closest_songs_indexes


In [ ]:
# Deep variational encoder

encoded_dim = 16

input_song = Input(shape = (df.shape[1], ))

encoded_h1 = Dense(64, activation = 'relu')(input_song)
encoded_h2 = Dense(32, activation = 'relu')(encoded_h1)
encoded = Dense(encoded_dim, activation = 'relu')(encoded_h2)

decoded_h1 = Dense(32, activation='relu')(encoded)
decoded_h2 = Dense(64, activation='relu')(decoded_h1)
decoded = Dense(df.shape[1], activation='sigmoid')(decoded_h2)

deep_autoencoder = Model(input_song, decoded) # techincally autoencoder_and_decoder

deep_encoder = Model(input_song, encoded) # Half of the autoencoder

deep_autoencoder.compile(optimizer='nadam', loss='mean_squared_error')

In [ ]:
stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3)
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

logdir = os.path.join("logs", f"SimpleAutoencoder-{now}")
tensorboard = TensorBoard(log_dir=logdir)

deep_autoencoder.fit(df, # input image to encoder
                df, # provide input image to decoder so the model learns how to reconstruct the input image 
                epochs=100,
                validation_split=.2,
                callbacks=[stop, tensorboard])

### Deep NN Summary
Val loss = 0.2960

In [ ]:
df.dtypes

In [ ]:
decoded_songs = autoencoder.predict(df)
encoded_songs = encoder.predict(df)

from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=11, algorithm='ball_tree')
nn.fit(encoded_songs)

In [ ]:
encoded_songs.shape

In [ ]:
# # Testing
# raw_df = pd.read_csv('./Data/tracks.csv')

# test_songs = {'California Love': '0LyLz6XsVs6wz85dK0S6EG', 'The Gambler': '5KqldkCunQ2rWxruMEtGh0', 
#               'Eye of the Tiger': '2HHtWyy5CgaQbC7XSoOb0e', 'You are my Sunshine': '4ApTr4wbyOucy2lbAvyuuV'}

# # Loop over all our test songs and get the recommendations
# recommendations = {}

# print('DeepSpot\'s recommendation system:\n\n')

# for name, id in test_songs.items():
#     query = raw_df.index[raw_df['id'] == id][0]
#     test_song = pd.DataFrame([raw_df.iloc[query, :]])
    
#     score, close_image_idx = nn.kneighbors(encoded_songs[query].reshape(1,-1))
#     k_closest_songs_indexes = close_image_idx[0][1:]
    
#     print('Top 10 recommendations for {} by {}:\n'.format(name, test_song.artists))
#     for i in range(len(k_closest_songs_indexes)):
#         song = raw_df.iloc[k_closest_songs_indexes[i], :]
#         print('{} by {} with a score of {}\n'.format(song['name'], song.artists, score[0][1:][i]))
#         print('URL: https://open.spotify.com/track/{}\n'.format(song['id']))
#     print('___________________________________________________________________________________\n')
    
#     recommendations[name] = k_closest_songs_indexes



# Training a Variational Autoencoder

In [14]:
# Variational Autoencoder
from tensorflow.keras import layers

class Sampling(layers.Layer):
    """
    Uses (z_mean, z_log_var) to sample a z vector from the latent N-dimensional Normal distribution
    i.e. Z is the vector encoding a digit.
    """
    def call(self, inputs):
        
        # recall from the video that z_mean and z_log_var are vectors 
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        
        # sample from an N-dimensional normal distribution
        # epsilon is given shape (batch, dim) because we are adding it to z_mean which has shape (batch, dim)
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        
        # output of tf.exp() is a vector 
        simga = tf.exp(0.5 * z_log_var)
        
        # this is our hideen latent vector made up of a mean and variance vector 
        # variance vector is scaled by epsilon, which is sampled from a normal distribtuion 
        # the the video guy said "stocastic" in reference to epsilon, he meant random 
        Z = z_mean + simga * epsilon
        
        # return hidden latent vector 
        return Z

In [19]:
df = pd.read_csv('../../songs_cleaned_autoencoder_8_24.csv', index_col='Unnamed: 0')

In [20]:
df.head(2)

,popularity,duration_ms,explicit,danceability,energy,key_1,key_2,key_3,key_4,key_5,...,key_12,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,-1.153128,-0.765231,-0.218921,0.476586,-0.390370,1,0,0,0,0,...,0,-0.593681,1,1.669464,0.637973,2.373142,-0.361517,-1.670382,-0.457098,-1.772010
1,-1.480897,-0.978657,-0.218921,0.776863,-1.112757,1,0,0,0,0,...,0,-2.308889,1,4.199676,0.990898,-0.422701,-0.377335,0.383636,-0.552432,-5.847677


In [21]:
encoded_dim = 16

# recall from the video that the more dimensions that our latent vector has
# the better the results of our model 
latent_dim = 8

# shape of our input data
# we are creating our input layer using Keras's Input() class
# the only thing that input layers really do is define the dimensionality of the input data for the model
encoder_inputs = Input(shape=(df.shape[1], ))

# these are the hidden layers

# pass data vector into FCFF layer 
encoded = Dense(encoded_dim, activation = 'relu')(encoder_inputs)

# recall that ordinarly the output of the last encoding layer is the latent vector 
# but here we are creating two output layers for our encoder - one for the mean and one for the log variance 
# returns a 2-dim mean vector
z_mean = Dense(latent_dim, name="z_mean")(encoded)
# returns a 2-dim log variance vecotr 
z_log_var = Dense(latent_dim, name="z_log_var")(encoded)
# pass mean and variance vector into Sampling class in order to create the Z vector,  Z = mean + var * epsilon
z = Sampling()([z_mean, z_log_var])

# ok, now let's put it all together 
# this is our encoder model 
# inputs are the original images
# outputs are the Z vectors: mean, log variance, and the complete Z, i.e. vector Z = mean + var * epsilon
encoder = Model(inputs=encoder_inputs, outputs=[z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 26)]         0                                            
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 16)           432         input_4[0][0]                    
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 8)            136         dense_2[0][0]                    
__________________________________________________________________________________________________
z_log_var (Dense)               (None, 8)            136         dense_2[0][0]                    
____________________________________________________________________________________________

In [22]:
# the input layer to our decoder has the same dimensionality as the latent vector
# because the latent vector is the input to the decoder model
latent_inputs = Input(shape=(latent_dim,))

# these are the hidden layers of our decoder 
# the data at this point is in a vector, the Z latent vector 

# this is the final layer in out decoder
# therefore this layer outputs the reconstruction of the original image 
decoder_outputs = Dense(df.shape[1], activation='sigmoid')(latent_inputs)

# this is our decoder model 
decoder = Model(inputs=latent_inputs, outputs=decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
dense_3 (Dense)              (None, 26)                234       
Total params: 234
Trainable params: 234
Non-trainable params: 0
_________________________________________________________________


In [23]:
from tensorflow import keras

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        """
        This class build a Variational Auto-Encoder. It accepts an Encoder and Decoder model as input. 
        
        Note
        ----
        This VAE class is inheriting Keras's Model API so that it can use the Model class methods 

        """
        # how python 3 handels inheritance 
        super(VAE, self).__init__(**kwargs)
        # set encoder model as class attribute
        self.encoder = encoder
        # set decoder model as class attribute 
        self.decoder = decoder
        # set mean function as class attribute - this calculates the total loss
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        # set mean function as class attribute - this calculates the reconstruction loss
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        # set mean function as class attribute - this calculates the kl loss
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        """
        Returns all loses in a list
        """
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        """
        Training our model via gradient descent and back-propagation 
        """
        
        # we used tf.GradientTape() in Sprint 2 Module 2 to run Gradient Descent from scratch 
        with tf.GradientTape() as tape:
            # pass input data into encoder model 
            # output of encoder model is the hidden state distribution parameters and hidden state vector 
            z_mean, z_log_var, z = self.encoder(data)
            
            # pass hidden state vector into decoder model 
            reconstruction = self.decoder(z)
            
            # calculate the reconstruction loss 
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.mse(data, reconstruction)#, axis=(1, 2)
                )
            )
            
            # calculate the kl loss
            #                (1 + z_simga   - (z_mean)^2        - e^(z_simga) ) 
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            # recall that we used tf.reduce_sum() in Sprint 2 Module 4 assignment 
            # it takes the sum of the vector components 
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            
            # calculate the total loss by adding the two loss components 
            total_loss = reconstruction_loss + kl_loss
            
        # now that we have calculated the loss function, we can perform Gradient Descent
        # we are passing in the loss function and the weights that we want to update via Gradeint Descent 
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        
        # log the total loss
        self.total_loss_tracker.update_state(total_loss)
        
        # log the reconstruction loss 
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        
        # log the kl loss 
        self.kl_loss_tracker.update_state(kl_loss)
        
        # return all the losses in a dictionary 
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [24]:
# instantiate a Variational Auto-Encoder model 
vae = VAE(encoder, decoder)

# complie the model 
vae.compile(optimizer=keras.optimizers.Nadam())

stop = EarlyStopping(monitor='loss', min_delta=0.001, patience=3, restore_best_weights=True)
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

logdir = os.path.join("logs", f"VAE-{now}")
tensorboard = TensorBoard(log_dir=logdir)

# train the model weights 
vae.fit(df, epochs=100,
        callbacks=[stop, tensorboard])

ValueError: Please provide as model inputs either a single array or a list of arrays. You passed: inputs=        popularity  duration_ms  explicit  danceability    energy  key_1  \
0        -1.153128    -0.765231 -0.218921      0.476586 -0.390370      1   
1        -1.480897    -0.978657 -0.218921      0.776863 -1.112757      1   
2        -1.480897    -0.358223 -0.218921     -0.790583 -1.454105      0   
3        -1.480897    -0.393417 -0.218921     -1.469210 -1.781164      0   
4        -1.480897    -0.496230 -0.218921     -0.982761 -1.529520      0   
...            ...          ...       ...           ...       ...    ...   
446470    2.124566    -0.331381 -0.218921     -0.021874 -2.025268      0   
446471    1.250514     0.211551 -0.218921     -0.033885 -0.100621      1   
446472    2.452335    -0.569003 -0.218921      1.197251  0.474908      1   
446473    2.343079    -0.313899 -0.218921     -0.184024 -0.910330      0   
446474    1.687540    -0.652952 -0.218921      0.782869  0.

# VAE Summary:
loss: 15.5321 - reconstruction_loss: 13.6563 - kl_loss: 1.9151

In [ ]:
# Encoding our songs
z_mean, z_log_var, z = vae.encoder.predict(df)

embeddings_df = pd.DataFrame(z, index=df.index)

# fitting KNN

nn = NearestNeighbors(n_neighbors=11, algorithm='ball_tree')
nn.fit(z)

In [ ]:
# Testing the VAE - It seems to work!
raw_df = pd.read_csv('./Data/tracks.csv')

test_songs = {'California Love': '0LyLz6XsVs6wz85dK0S6EG', 'The Gambler': '5KqldkCunQ2rWxruMEtGh0', 
              'Eye of the Tiger': '2HHtWyy5CgaQbC7XSoOb0e', 'You are my Sunshine': '4ApTr4wbyOucy2lbAvyuuV'}

# Loop over all our test songs and get the recommendations
recommendations = {}

print('DeepSpot\'s recommendation system:\n\n')

for name, id in test_songs.items():
    query = raw_df.index[raw_df['id'] == id][0]
    test_song = pd.DataFrame([raw_df.iloc[query, :]])
    
    score, close_image_idx = nn.kneighbors(z[query].reshape(1,-1))
    k_closest_songs_indexes = close_image_idx[0][1:]
    
    print('Top 10 recommendations for {} by {}:\n'.format(name, test_song.artists))
    for i in range(len(k_closest_songs_indexes)):
        song = raw_df.iloc[k_closest_songs_indexes[i], :]
        print('{} by {} with a score of {}\n'.format(song['name'], song.artists, score[0][1:][i]))
        print('URL: https://open.spotify.com/track/{}\n'.format(song['id']))
    print('___________________________________________________________________________________\n')
    
    recommendations[name] = k_closest_songs_indexes




# Hyperparameter tuning the VAE

In [ ]:
latent_dim = 8

encoder_inputs = Input(shape=(df.shape[1], ))

h1 = Dense(16, activation = 'relu')(encoder_inputs)
encoded = Dense(encoded_dim, activation = 'relu')(h1)

z_mean = Dense(latent_dim, name="z_mean")(encoded)

z_log_var = Dense(latent_dim, name="z_log_var")(encoded)

z = Sampling()([z_mean, z_log_var])

encoder = Model(inputs=encoder_inputs, outputs=[z_mean, z_log_var, z], name="encoder")
encoder.summary()

In [ ]:
latent_inputs = Input(shape=(latent_dim,))

dh1 = Dense(16, activation='relu')(latent_inputs)

decoder_outputs = Dense(df.shape[1], activation='sigmoid')(dh1)

# this is our decoder model 
decoder = Model(inputs=latent_inputs, outputs=decoder_outputs, name="decoder")
decoder.summary()

In [ ]:
# instantiate a Variational Auto-Encoder model 
vae = VAE(encoder, decoder)

# complie the model 
vae.compile(optimizer=keras.optimizers.Nadam())

stop = EarlyStopping(monitor='loss', min_delta=0.001, patience=3, restore_best_weights=True)
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

logdir = os.path.join("logs", f"DeepVAE-{now}")
tensorboard = TensorBoard(log_dir=logdir)

# train the model weights 
vae.fit(df, epochs=100,
        callbacks=[stop, tensorboard])

Results deep VAE v1 (latent_dim = 16, h1 = 32): 
loss: 15.9362 - reconstruction_loss: 14.5233 - kl_loss: 1.4098

**Results deep VAE v2 (latent_dim = 8, h1 = 16):**
Epoch 21/100
loss: 15.4746 - reconstruction_loss: 13.8923 - kl_loss: 1.6215

Results regular VAE v1 (latent_dim = 16): 
loss: 15.5321 - reconstruction_loss: 13.6563 - kl_loss: 1.9151

Results regular VAE v1 (latent_dim = 8):
Epoch 12/100
loss: 15.5430 - reconstruction_loss: 13.7091 - kl_loss: 1.8810

In [ ]:
# Testing results
# Encoding our songs
z_mean, z_log_var, z = vae.encoder.predict(df)

embeddings_df = pd.DataFrame(z, index=df.index)

# fitting KNN

nn = NearestNeighbors(n_neighbors=11, algorithm='ball_tree')
nn.fit(z)

# Testing the VAE v2
raw_df = pd.read_csv('./Data/tracks.csv')

test_songs = {'California Love': '0LyLz6XsVs6wz85dK0S6EG', 'The Gambler': '5KqldkCunQ2rWxruMEtGh0', 
              'Eye of the Tiger': '2HHtWyy5CgaQbC7XSoOb0e', 'You are my Sunshine': '4ApTr4wbyOucy2lbAvyuuV'}

# Loop over all our test songs and get the recommendations
recommendations = {}

print('DeepSpot\'s recommendation system:\n\n')

for name, id in test_songs.items():
    query = raw_df.index[raw_df['id'] == id][0]
    test_song = pd.DataFrame([raw_df.iloc[query, :]])
    
    score, close_image_idx = nn.kneighbors(z[query].reshape(1,-1))
    k_closest_songs_indexes = close_image_idx[0][1:]
    
    print('Top 10 recommendations for {} by {}:\n'.format(name, test_song.artists))
    for i in range(len(k_closest_songs_indexes)):
        song = raw_df.iloc[k_closest_songs_indexes[i], :]
        print('{} by {} with a score of {}\n'.format(song['name'], song.artists, score[0][1:][i]))
        print('URL: https://open.spotify.com/track/{}\n'.format(song['id']))
    print('___________________________________________________________________________________\n')
    
    recommendations[name] = k_closest_songs_indexes

embeddings_df.to_csv('./Data/vae_embeddings.csv')



In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
autoencoder.save('saved_model/vae_002')
encoder.save('saved_model/encoder_002')